In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import torch

c:\Users\sudos\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/sudoghut/contradictory-my-dear-watson/main/data/train_10.csv')
print(df_raw.head(5))
print(df_raw["label"].value_counts())


           id                                            premise  \
0  5130fd2cb5  and these comments were considered in formulat...   
1  5b72532a0b  These are issues that we wrestle with in pract...   
2  3931fbe82a  Des petites choses comme celles-là font une di...   
3  5622f0c60b  you know they can't really defend themselves l...   
4  86aaa48b45  ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...   

                                          hypothesis lang_abv language  label  
0  The rules developed in the interim were put to...       en  English      0  
1  Practice groups are not permitted to work on t...       en  English      2  
2              J'essayais d'accomplir quelque chose.       fr   French      0  
3  They can't defend themselves because of their ...       en  English      0  
4    เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร       th     Thai      1  
0    5
2    2
1    2
Name: label, dtype: int64


In [13]:
df_raw["premise"]

0    and these comments were considered in formulat...
1    These are issues that we wrestle with in pract...
2    Des petites choses comme celles-là font une di...
3    you know they can't really defend themselves l...
4    ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...
5    Bir çiftlikte birisinin, ağıla kapatılmış bu ö...
6    ریاست ہائے متحدہ امریکہ واپس آنے پر، ہج ایف بی...
7                From Cockpit Country to St. Ann's Bay
8    Look, it's your skin, but you're going to be i...
Name: premise, dtype: object

In [17]:
# tokenizer: https://huggingface.co/bert-base-multilingual-cased
# tokenizer idea from: https://www.kaggle.com/code/anasofiauzsoy/tutorial-notebook/notebook

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = df_raw["premise"][4]
encoded_input = tokenizer(text, return_tensors='pt')
# output = model(**encoded_input)
print(df_raw["premise"][4])
print(encoded_input)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสดงออกและได้เล่นหลายบทบาทไปพร้อมกัน ๆ อาจช่วยให้เด็กจับความคล้ายคลึงและความแตกต่างระหว่างผู้คนในด้านความปรารถนา ความเชื่อ และความรู้สึกได้
{'input_ids': tensor([[   101,  46301,  53123, 111431,  20507,  34523,  49097,  37022,  49097,
         107854,  28767,  17405,  17405,  53936,  80275,  18427, 111437, 111431,
          42407,  34523,  89969,   1452,  63667,  17344,  28767,  18203,  46856,
          22598, 111435,  81831, 111432,  28767,  82275,  33178,  63667, 111432,
          20507,  22598, 111435,  81831, 111431,  20507,  34523, 111424,  61888,
          20503,  49097,  37022,  49097, 107854, 111435,  49292,  39901,  22765,
          35933,  33178,  17405,  89969,   1455,   1436,  17344,  46856,  42407,
          31904,  31287,  20503, 111434, 111424,  35933, 111431,  22123, 111437,
          18427,  46856, 102269,  31256,  31287,  79790,  31256,  20507,  43102,
          20503,  31256,  20507, 111428,  19197, 111432,  20507,  22598,  